## Redshift Setup with Python SDK (boto3)
This notebook will show how to set up some AWS resources using the Python SDK for AWS, boto3.

In [2]:
import boto3


SyntaxError: invalid syntax (284038561.py, line 2)

In [31]:
#Custom code for importing AWS credentials from where VS Code stores them on local machine to a local Python file,
#which can be more easily imported as credentials to Python (https://stackoverflow.com/questions/25501403/storing-the-secrets-passwords-in-a-separate-file)

'''
Note: Deprecated. Could be a cool method to store credentials in the future, but not worth creating a whole new file for
just these credentials (less secure too).
'''
#import shutil

# #Find and copy file
# orig_path = "/home/rambino/.aws/credentials"
# #Must be full file path to ensure .gitignore understands it
# new_file_name = "cred_*.py"
# new_file_name = os.getcwd() + "/" + new_file_name

# shutil.copyfile(orig_path,new_file_name)

# #add to local .gitignore (credentials):
# with open("../.gitignore","a") as file:
#     file.write("\n")
#     file.write(new_file_name)


In [53]:
#Probably over-complicated function for extracting text from a file and matching it to a key for later use in code.
#Using it here for relatively safe way of getting AWS credentials from original VS Code-created directory

import re
def searchAndExtract(file_path,extracts):
    result = {}

    with open(file_path,"r") as file:
        for line in file:
            for key in extracts.keys():
                match_obj = extracts[key].match(line)
                if(match_obj):
                    result[key] = match_obj.group(1)
    
    return result 


dict_keys(['key', 'secret'])

In [ ]:
#Getting AWS Credentials

orig_path = "/home/rambino/.aws/credentials"

extracts = {
    "key":re.compile("aws_access_key_id = (.*)"),
    "secret":re.compile("aws_secret_access_key = (.*)")
}

aws_cred = searchAndExtract(orig_path,extracts)

In [62]:
#Getting RedShift username + password
orig_path = "./redshift_credentials.txt"

extracts = {
    "user":re.compile("UN: (.*)"),
    "password":re.compile("PW: (.*)")
}

redshift_cred = searchAndExtract(orig_path,extracts)

In [67]:
iam = boto3.client('iam',
    region_name="us-west-2",
    aws_access_key_id=aws_cred['key'],
    aws_secret_access_key=aws_cred['secret']
)

In [71]:
#Create IAM role:

#This policy is something about allowing Redshift to impersonate a user, but I don't really understand it.
#Look more into what "sts:AssumeRole" really means.

import json

dwhRole = iam.create_role(
    Path = "/",
    RoleName =  "RedShift_Impersonation",
    Description = "Allows redshift to access S3",
    AssumeRolePolicyDocument=json.dumps(
        {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": 'sts:AssumeRole',
                    "Principal":{"Service": "redshift.amazonaws.com"}
                }
            ]
        }
    )
)

dwhRole

In [73]:
#Attaching IAM policy to the role (which actually gives permissions):
attach_response = iam.attach_role_policy(
    RoleName = "RedShift_Impersonation",
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
)

attach_response

{'ResponseMetadata': {'RequestId': 'adff6259-83f2-4371-8a63-2c18104d452d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'adff6259-83f2-4371-8a63-2c18104d452d',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Sun, 14 Aug 2022 18:56:57 GMT'},
  'RetryAttempts': 0}}

In [56]:
redshift = boto3.client('redshift',
    region_name="us-west-2",
    aws_access_key_id=aws_cred['key'],
    aws_secret_access_key=aws_cred['secret']
)

In [64]:
#Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster
response = redshift.create_cluster(
    ClusterType = "multi-node",
    NodeType = 'dc2.large',
    NumberOfNodes = 4,
    DBName = "my_redshift_db",
    ClusterIdentifier = 'redshift-cluster-1',
    MasterUsername = redshift_cred["user"],
    MasterUserPassword = redshift_cred["password"],
    IamRoles = [dwhRole['Role']['Arn']]
)

'''
WARNING! After running this code, you WILL create a Redshift cluster. Be sure to delete it to not incur costs!!
'''

response

In [83]:
response = redshift.delete_cluster(
    ClusterIdentifier = 'redshift-cluster-1',
    SkipFinalClusterSnapshot=True
)

InvalidClusterStateFault: An error occurred (InvalidClusterState) when calling the DeleteCluster operation: There is an operation running on the Cluster. Please try to delete it at a later time.

In [84]:
redshift.describe_clusters()

{'Clusters': [],
 'ResponseMetadata': {'RequestId': 'b9f1f8b0-034b-46c6-a1f7-f926898387e0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b9f1f8b0-034b-46c6-a1f7-f926898387e0',
   'content-type': 'text/xml',
   'content-length': '287',
   'date': 'Sun, 14 Aug 2022 19:05:14 GMT'},
  'RetryAttempts': 0}}